In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re # Importiamo il modulo per le espressioni regolari
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

2025-09-20 16:23:24.870659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758378205.073980    4269 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758378205.172109    4269 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758378205.651300    4269 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758378205.651413    4269 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758378205.651420    4269 computation_placer.cc:177] computation placer alr

In [2]:
# Load data
df_path = 'Datasets/'
users = pd.read_csv(df_path + 'v2_top10_embeddings_users.csv')
emb = np.load(df_path + 'v2_top10_embeddings.npy')
df2 = pd.concat([users, pd.DataFrame(emb)], axis=1)

#load features
features = pd.read_csv(df_path + 'df_preprocessed.csv')
desc = np.load(df_path + 'description_embeddings.npy')

desc_merged = pd.concat([features, pd.DataFrame(desc)], axis=1)
desc_merged.rename(columns=lambda c: f"D{c}" if str(c).isdigit() else c, inplace=True)
df2['has_embedding'] = 1
df2.rename(columns=lambda c: f"T{c}" if str(c).isdigit() else c, inplace=True)
df2_merged = desc_merged.merge(df2, left_on="id", right_on='author_id',how="left")
df = df2_merged.fillna(0)
print(df.shape)
print(df['label_x'].value_counts())


/tmp/ipykernel_4269/3929193048.py:8: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv(df_path + 'df_preprocessed.csv')


(1000000, 1220)
label_x
human    860057
bot      139943
Name: count, dtype: int64


In [3]:

# 1. Inizializziamo le liste per i nomi delle colonne
d_cols = []
t_cols = []
other_cols = []

# Definiamo i pattern per la ricerca usando espressioni regolari
# ^D\d+$ : Cerca stringhe che iniziano (^) con 'D' seguita da uno o più numeri (\d+) fino alla fine ($)
d_pattern = re.compile(r'^D\d+$')
t_pattern = re.compile(r'^T\d+$')


# 2. Iteriamo su tutte le colonne del DataFrame
for col in df.columns:
    if d_pattern.match(col):
        d_cols.append(col)
    elif t_pattern.match(col):
        t_cols.append(col)
    else:
        other_cols.append(col)

d_cols.append('split_x')
d_cols.append('label_x')
t_cols.append('split_x')
t_cols.append('label_x')

print(f"Trovate {len(d_cols)} colonne che iniziano con 'D' e un numero.")
print(f"Trovate {len(t_cols)} colonne che iniziano con 'T' e un numero.")
print(f"Trovate {len(other_cols)} colonne restanti.")

# 3. Creiamo i tre nuovi DataFrame
df_d_features = df[d_cols]
df_t_features = df[t_cols]
df_other_features = df[other_cols]

print(df_d_features.shape, df_t_features.shape, df_other_features.shape)


Trovate 386 colonne che iniziano con 'D' e un numero.
Trovate 770 colonne che iniziano con 'T' e un numero.
Trovate 68 colonne restanti.
(1000000, 386) (1000000, 770) (1000000, 68)


In [4]:
def prepare_data(df, features=0):
    print(f"Dataset shape: {df.shape}")
    #print(f"Class distribution:\n{df['label'].value_counts()}")

    # Get feature columns (the numeric columns 0-383)
    #feature_cols = [i for i in range(df.shape[1]-50)]
    if features:
        cols_to_remove = ['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'account_age_days', 'author_id', 'label_y', 'split_y', 'withheld', 'n_tweets']
        df = df.drop(columns=cols_to_remove)
    df = df.rename(columns={'label_x': 'label', 'split_x':'split'})

    #X_emb = df[feature_cols]
    #X_features = df[df.columns.difference(cols_to_remove+feature_cols)]
    le = LabelEncoder()
    y = le.fit_transform(df['label'])
    print(df['label'][0], y[0])

    # Split using existing split column
    """X_train_emb = X_emb[df['split'] == 'train']
    X_train_features = X_features[df['split'] == 'train']
    X_train_features = X_train_features.drop(columns=['label', 'split'])"""
    X_train = df[df['split'] == 'train']
    X_train = X_train.drop(columns=['label', 'split'])
    y_train = y[df['split'] == 'train']
    """X_val_emb = X_emb[df['split'] == 'val']
    X_val_features = X_features[df['split'] == 'val']
    X_val_features = X_val_features.drop(columns=['label', 'split'])"""
    X_val = df[df['split'] == 'val']
    X_val = X_val.drop(columns=['label', 'split'])
    y_val = y[df['split'] == 'val']
    """X_test_emb = X_emb[df['split'] == 'test']
    X_test_features = X_features[df['split'] == 'test']
    X_test_features = X_test_features.drop(columns=['label', 'split'])"""
    X_test = df[df['split'] == 'test']
    X_test = X_test.drop(columns=['label', 'split'])
    y_test = y[df['split'] == 'test']

    print(f"\nSplit sizes - Train: {len(y_train)}, Val: {len(y_val)}, Test: {len(y_test)}")
    print(f"Train class distribution: {pd.Series(y_train).value_counts().to_dict()}")
    print(f"Validation class distribution: {pd.Series(y_val).value_counts().to_dict()}")
    print(f"Test class distribution: {pd.Series(y_test).value_counts().to_dict()}")

    return X_train, y_train, X_val, y_val, X_test, y_test, le

In [5]:
print("Preprocessing data...")
# --- Prepare Labels (y) ---
X_train_emb1, _, X_val_emb1, _, X_test_emb1, _, le = prepare_data(df_d_features)
X_train_emb2, _, X_val_emb2, _, X_test_emb2, _, le = prepare_data(df_t_features)
X_train_features, y_train, X_val_features, y_val, X_test_features, y_test, le = prepare_data(df_other_features, 1)

X_train = [X_train_features, X_train_emb1, X_train_emb2 ]
X_val = [X_val_features, X_val_emb1, X_val_emb2]
X_test = [X_test_features, X_test_emb1, X_test_emb2]

Preprocessing data...
Dataset shape: (1000000, 386)
human 1

Split sizes - Train: 700000, Val: 200000, Test: 100000
Train class distribution: {1: 645414, 0: 54586}
Validation class distribution: {1: 144087, 0: 55913}
Test class distribution: {1: 70556, 0: 29444}
Dataset shape: (1000000, 770)
human 1

Split sizes - Train: 700000, Val: 200000, Test: 100000
Train class distribution: {1: 645414, 0: 54586}
Validation class distribution: {1: 144087, 0: 55913}
Test class distribution: {1: 70556, 0: 29444}
Dataset shape: (1000000, 68)
human 1

Split sizes - Train: 700000, Val: 200000, Test: 100000
Train class distribution: {1: 645414, 0: 54586}
Validation class distribution: {1: 144087, 0: 55913}
Test class distribution: {1: 70556, 0: 29444}


In [6]:
def build_classifier_model(num_numerical_features, emb1_shape, emb2_shape):
    """
    Builds a multi-input Keras model for classification.
    """
    # Input layer for standard numerical features
    input_numeric = Input(shape=(num_numerical_features,), name='numerical_input')
    # Processing branch for numerical features
    x1 = Dense(64, activation='relu')(input_numeric)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(32, activation='relu')(x1)

    # Input layer for the first embedding
    input_emb1 = Input(shape=(emb1_shape,), name='embedding1_input')
    # (Optional) Add a dense layer to process the first embedding
    x2 = Dense(256, activation='relu')(input_emb1)
    x2 = Dropout(0.3)(x2)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dropout(0.3)(x2)
    x2 = Dense(64, activation='relu')(x2)

    # Input layer for the second embedding
    input_emb2 = Input(shape=(emb2_shape,), name='embedding2_input')
    # (Optional) Add a dense layer to process the second embedding
    x3 = Dense(512, activation='relu')(input_emb2)
    x3 = Dropout(0.3)(x3)
    x3 = Dense(256, activation='relu')(x3)
    x3 = Dropout(0.3)(x3)
    x3 = Dense(128, activation='relu')(x3)

    # Concatenate all processed inputs
    concatenated = concatenate([x1, x2, x3])

    # Final classification layers
    combined = Dense(128, activation='relu')(concatenated)
    combined = Dropout(0.5)(combined)
    combined = Dense(64, activation='relu')(combined)
    output = Dense(1, activation='sigmoid', name='output')(combined) # Sigmoid for binary classification

    # Create and compile the model
    model = Model(inputs=[input_numeric, input_emb1, input_emb2], outputs=output)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', AUC()])
    return model

In [7]:
# --- Build the Model ---
print("\n3. Building the model...")
model = build_classifier_model(
    num_numerical_features=X_train_features.shape[1],
    emb1_shape=X_train_emb1.shape[1],
    emb2_shape=X_train_emb2.shape[1]
)
model.summary()


3. Building the model...


2025-09-20 16:24:46.749502: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ embedding1_input    │ (None, 384)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding2_input    │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     98,560 │ embedding1_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 512)       │    393,728 │ embedding2_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numerical_input     │ (None, 47)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 512)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      3,072 │ numerical_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256)       │    131,328 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 256)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128)       │     32,896 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 224)       │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense_4[0][0],    │
│                     │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 128)       │     28,800 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 128)       │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │      8,256 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ dense_9[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 739,937 (2.82 MB)

 Trainable params: 739,937 (2.82 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# --- Train the Model ---
print("\n4. Training the model...")
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val), # Use 20% of training data for validation
    callbacks=[early_stopping],
    class_weight=class_weights,
    verbose=1
)


4. Training the model...


In [105]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)
print(y_test[:10], y_pred_probs[:10])
print(classification_report(y_test, y_pred, target_names=['bot', 'human']))

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
[1 1 0 0 0 1 0 1 1 1] [[0.79819834]
 [0.7609072 ]
 [0.7640944 ]
 [0.7609072 ]
 [0.7609072 ]
 [0.8094242 ]
 [0.7609072 ]
 [0.7609072 ]
 [0.7609072 ]
 [0.7663691 ]]
              precision    recall  f1-score   support

         bot       0.21      0.00      0.00     29444
       human       0.71      1.00      0.83     70556

    accuracy                           0.70    100000
   macro avg       0.46      0.50      0.41    100000
weighted avg       0.56      0.70      0.58    100000

